# Apex Tracking - Data Collection

In this notebook, we'll collect a dataset that will be used to steer JetRacer.  To do this, we'll manually place JetRacer in different positions and orientations along the track, and collect a dataset of x, y coordinates that correspond to a target point that JetRacer should steer torwards.  Taking some liberties, we'll call this target point the *apex*.

To collect this dataset, follow these steps

1.  Place JetRacer in a random position / orientation on the track

2.  Imagine the optimal path that JetRacer would follow

3.  Click as far along that imaginary path such that JetRacer could travel straight there without running out of bounds (or hitting an object, etc.)

4.  Repeat 1-3

We've been able to get a decent working neural network using about 100 images following this procedure.  If you're familiar with our previous project, *JetBot*, you'll know that it's not only important to have a good amount of images, but well varied images.

Make sure you cover different spots on the track, different offsets from the centerline, and if you can vary other parameters like brightness.

After we've collected a dataset to test, we'll open up the *train_model.ipynb* notebook to train our neural network.  

So let's get started.  First, we'll create and display a *ClickableImageWidget*, which will be used to annotate the images.

In [ ]:
from IPython.display import display
from jupyter_clickable_image_widget import ClickableImageWidget

preview_widget = ClickableImageWidget(format='jpeg', width=224, height=224)

display(preview_widget)

Right now, you won't see any image displayed.  We need to initialize and connect the camera to the widget. Let's start our camera and connect it to the clickable image widget using the *traitlets* library.

The IMX219 camera driver on Jetson supports up to 120FPS at certain capture resolutions.  One of these capture resolutions is 1280x720, so we will manually set that.  Our output resolution will be 224x224, which will be fed directly to the neural network. Since we're only collecting data, we will reduce the framerate to just 10FPS to limit network streaming bandwidth.

We set the ``camera.running = True`` so that the camera's internal ``value`` will be set automatically.  This allows us to attach callbacks to it which will automatically get called when new camera frames arrive

In [ ]:
import traitlets
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=10)

camera.running = True

camera_link = traitlets.dlink((camera, 'value'), (preview_widget, 'value'), transform=bgr8_to_jpeg)

Now, let's create another image widget that will be used to display the latest snapshot sample that we add to our dataset.

We'll also create a widget to display the number of images in the dataset.

In [ ]:
import ipywidgets

snapshot_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
count_widget = ipywidgets.IntText(description='count')

display(snapshot_widget, count_widget)

This won't display anything yet.  We need to create the dataset class, and attach a callback to the preview *ClickableImageWidget* to save the clicked x, y coordinates.

We've created two python files in the ``apex_tracking`` example folder named ``xy_dataset.py`` and ``utils.py``.  ``xy_dataset.py`` defines a class that makes it easy for us to save / load annotations, which we'll also use for training.  Under the hood, this dataset just dumps images in a folder with the name ``<x_coordinate>_<y_coordinate>_<uuid>.jpg``.  Where ``uuid`` is a unique string that is generated using the Python ``uuid`` package.

In [ ]:
from xy_dataset import XYDataset

dataset = XYDataset('apex_dataset')

We can add (image, x, y) tuples to the dataset by calling the ``save_entry`` method.  We'll do this in a callback that we attach to the ClickableImageWidget below.

In [ ]:
import cv2

def save_snapshot(_, content, msg):
    global camera, dataset, snapshot_widget
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        dataset.save_entry(camera.value, x, y)
        
        # draw circle on snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
    
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(dataset)
        
preview_widget.on_msg(save_snapshot)

Now, if we click the preview widget, we'll see an image drawn on the snapshot widget with a green circle.  We should also see the image count increase.

To make it easier, let's display all of the widgets below.

In [ ]:
display(
    ipywidgets.HBox([preview_widget, snapshot_widget]),
    count_widget
)

Now go ahead and place JetRacer in different spots on the track and collect a dataset of at least 100 images as described earlier!

Remember to vary the placement of the racecar for each image.

In [ ]:
!zip -rq apex_dataset.zip apex_dataset

## Next

Follow the train_model.ipynb to train the neural network to predict the Apex from an image!